In [ ]:

import logging
from pathlib import Path

import lightning as L
import numpy as np
import torch as th
from lightning.pytorch import seed_everything
from lightning.pytorch.callbacks import  ModelCheckpoint
from torch.utils.data import DataLoader
from torchmetrics.classification.f_beta import F1Score
from torchmetrics import MetricCollection

from mamkit.configs.base import ConfigKey
from mamkit.configs.text import TransformerConfig
from mamkit.data.collators import UnimodalCollator, TextTransformerCollator
from mamkit.data.datasets import MMUSEDFallacy, InputMode
from mamkit.data.processing import UnimodalProcessor
from mamkit.models.text import Transformer
from mamkit.utility.callbacks import PycharmProgressBar
from mamkit.utility.model import MAMKitLightingModel


In [ ]:

logging.basicConfig(level=logging.INFO)
save_path = Path('results/mmused-fallacy/afc/text_only_RoBERTa')
save_path.mkdir(parents=True, exist_ok=True)
base_data_path = Path.cwd().joinpath('data')


In [ ]:

loader = MMUSEDFallacy(task_name='afc',
                       input_mode=InputMode.TEXT_ONLY,
                       base_data_path=base_data_path)

config = TransformerConfig.from_config(key=ConfigKey(dataset='mmused-fallacy',
                                                     task_name='afc',
                                                     input_mode=InputMode.TEXT_ONLY,
                                                     tags={'anonymous', 'roberta'}))

trainer_args = {
    'accelerator': 'auto',
    'devices': 1,
    'batch_size': 8,
    'max_epochs': 3,
}


config.optimizer = th.optim.Adam
config.optimizer_args = {'lr': 2e-5}

metrics = {}

In [ ]:

for seed in config.seeds:
    seed_everything(seed=seed)
    for split_info in loader.get_splits(key='mm-argfallacy-2025'):
        processor = UnimodalProcessor()
        processor.fit(split_info.train)
        split_info.train = processor(split_info.train)
        split_info.val = processor(split_info.val)
        split_info.test = processor(split_info.test)
        processor.clear()

        unimodal_collator = UnimodalCollator(
            features_collator=TextTransformerCollator(model_card=config.model_card,
                                                      tokenizer_args=config.tokenizer_args),
            label_collator=lambda labels: th.tensor(labels)
        )

        train_loader = DataLoader(split_info.train, batch_size=config.batch_size, shuffle=True, collate_fn=unimodal_collator)
        val_loader = DataLoader(split_info.val, batch_size=config.batch_size, shuffle=False, collate_fn=unimodal_collator)
        test_loader = DataLoader(split_info.test, batch_size=config.batch_size, shuffle=False, collate_fn=unimodal_collator)

        model = Transformer(model_card=config.model_card,
                            is_transformer_trainable=config.is_transformer_trainable,
                            head=config.head)
        model = MAMKitLightingModel(model=model,
                                    loss_function=config.loss_function,
                                    num_classes=config.num_classes,
                                    optimizer_class=config.optimizer,
                                    val_metrics=MetricCollection({'f1': F1Score(task='multiclass', num_classes=6)}),
                                    test_metrics=MetricCollection({'f1': F1Score(task='multiclass', num_classes=6)}),
                                    **config.optimizer_args)

        trainer = L.Trainer(**trainer_args,
                            callbacks=[ModelCheckpoint(monitor='val_loss', mode='min'),
                                       PycharmProgressBar()])
        trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)

        val_metrics = trainer.test(ckpt_path='best', dataloaders=val_loader)[0]
        test_metrics = trainer.test(ckpt_path='best', dataloaders=test_loader)[0]
        logging.info(f'Validation metrics: {val_metrics}')
        logging.info(f'Test metrics: {test_metrics}')

        for metric_name, metric_value in val_metrics.items():
            metrics.setdefault('validation', {}).setdefault(metric_name, []).append(metric_value)
        for metric_name, metric_value in test_metrics.items():
            metrics.setdefault('test', {}).setdefault(metric_name, []).append(metric_value)
        processor.reset()


In [ ]:

metric_names = list(metrics['validation'].keys())
for split_name in ['validation', 'test']:
    for metric_name in metric_names:
        metric_values = np.array(metrics[split_name][metric_name]).reshape(len(config.seeds), -1)
        per_seed_avg = metric_values.mean(axis=-1)
        per_seed_std = metric_values.std(axis=-1)
        avg = per_seed_avg.mean(axis=-1)
        std = per_seed_avg.std(axis=-1)
        metrics[split_name][f'per_seed_avg_{metric_name}'] = (per_seed_avg, per_seed_std)
        metrics[split_name][f'avg_{metric_name}'] = (avg, std)
logging.info(metrics)
np.save(save_path.joinpath('metrics.npy').as_posix(), metrics)
